# **HOW TO TRAIN NEW MODEL (En to Vi)**



Phiên bản V3 nhằm chạy thử model TED anh - việt với các đặc điểm:

1 - Chạy  1 epoch

2 - Thực hiện trên máy local

3 - Tiền xử lý chỉ có lower case mà thôi, sau đó là áp dụng bpe


## Data Pre-processing

In [9]:
# # Download and prepare the data
!bash examples/translation/prepare-en-vi-iwslt-only-BPE.sh

Cloning Moses github repository (for tokenization scripts)...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
Cloning Subword NMT repository (for BPE pre-processing)...
fatal: destination path 'subword-nmt' already exists and is not an empty directory.
--2020-12-02 10:17:50--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2020-12-02 10:17:51--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connect

# Training

In [12]:
!mkdir -p "bpe-iwslt-checkpoints/fconv_wmt_en_vi"

!CUDA_VISIBLE_DEVICES=0 fairseq-train "bpe-iwslt-data-bin/tokenized.en-vi" \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir "bpe-iwslt-checkpoints/fconv_wmt_en_vi" \
    --max-epoch 1


8, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_activations=False, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='bpe-iwslt-data-bin/tokenized.en-vi', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=

In [6]:
!pwd

/home/chinh/clc_fairseq


In [13]:
# Evaluate
# !mkdir -p iwslt-data-bin-v2/fconv_wmt_en_vi
!fairseq-generate \
    bpe-iwslt-data-bin/tokenized.en-vi/ \
    --path bpe-iwslt-checkpoints/fconv_wmt_en_vi/checkpoint_best.pt \
    --beam 5 --remove-bpe | tee bpe-iwslt-checkpoints/fconv_wmt_en_vi/gen.out

việc với những người đàn ông ta , và tôi đã làm việc với những người đàn ông ta .
P-149	-2.2725 -1.0182 -2.7739 -0.6407 -3.7880 -2.2473 -3.0627 -0.8222 -1.3618 -2.6074 -1.9664 -4.4139 -0.2124 -2.3271 -3.2057 -2.2480 -2.4978 -2.8015 -2.9333 -1.4247 -2.1668 -2.4235 -1.6645 -4.5058 -0.1738 -2.5959 -4.0623 -0.3142
S-17	and the answer , i believe , is yes . &#91; &quot; f = t <unk> s<unk> &quot; &#93; what you &apos;re seeing is probably the closest equivalent to an e = mc <unk> for intelligence that i &apos;ve seen .
T-17	tôi tin , câu trả lời là có . &#91; &quot; f = t <<unk>> s<<unk>> &quot; &#93; cái ta thấy đây có thể là cái tương đương gần nhất với một e = mc ² một công thức cho trí thông minh mà tôi đã thấy .
H-17	-2.2851414680480957	&lt; speaker &gt;
D-17	-2.2851414680480957	&lt; speaker &gt;
P-17	-5.0194 -0.0419 -1.2187 -2.8606
S-672	however , in every religion , in every population , you &apos;ll find a small percentage of people who hold so fervently to their beliefs that they fe

Compute BLEU score


In [14]:
!grep ^H bpe-iwslt-checkpoints/fconv_wmt_en_vi/gen.out | cut -f3- > gen.out.sys
!grep ^T bpe-iwslt-checkpoints/fconv_wmt_en_vi/gen.out | cut -f2- > gen.out.ref
!fairseq-score --sys gen.out.sys --ref gen.out.ref


Namespace(ignore_case=False, order=4, ref='gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='gen.out.sys')
BLEU4 = 1.10, 30.7/5.9/1.2/0.2 (BP=0.409, ratio=0.528, syslen=13859, reflen=26244)


BLEU4 = 1.1, thấp kỷ lục